In [1]:
#Requesting necessory packages to local
import requests
import os
import zipfile
import glob
from os import rename, listdir
import string
import sqlite3
import csv
import pandas as pd
from pandas import DataFrame
from pandas import ExcelWriter

#Creating staging folder in root path

staging_dir_name = "staging"
os.mkdir(staging_dir_name)

#Retriving and extracting necessory data fron the url into staging folder

url = "https://data.medicare.gov/views/bg9k-emty/files/0a9879e0-3312-4719-a1db-39fd114890f1?content_type=application%2Fzip%3B%20charset%3Dbinary&filename=Hospital_Revised_Flatfiles.zip"
r = requests.get(url)
zip_file_name = os.path.join(staging_dir_name,"Hospital_Revised_Flatfiles.zip")
zf = open(zip_file_name,"wb")
zf.write(r.content)
zf.close()
z = zipfile.ZipFile(zip_file_name, "r")
z.extractall(staging_dir_name)
z.close()

#Defining cleaning funtion for file names and column names

def format_data(x) :
    a = x.lower()
    b = a.replace(' ','_')
    c = b.replace('-','_')
    d = c.replace('%','pct')
    e = d.replace('/','_')
    rng = list(string.ascii_lowercase)
    if(e[0] in rng):
        opt = e;
    else :
        opt = 'c_'+e[0:]
    return opt
#Defining funtion for datatype concatenation in insert query 

def concat(x):
    return x+' text'

#Removing quotes from column and data items

def removequotes(x):
    z=x.replace("'",'')
    return (z)


cwd = os.chdir(os.getcwd()+'/staging')
list_files = glob.glob("*.csv")

#Removing corrupted files and cleaning data

if('FY2015_Percent_Change_in_Medicare_Payments.csv' in list_files) :
    os.remove('FY2015_Percent_Change_in_Medicare_Payments.csv')
    list_files.remove('FY2015_Percent_Change_in_Medicare_Payments.csv')

list_files = glob.glob("*.csv")
for lname in list_files:
    rename(lname, lname.replace(lname,format_data(lname)))
list_files = glob.glob("*.csv")

#Changing encoding of files

for enc in list_files:
    inp_files = open(enc,"rt", encoding='cp1252')
    i_data = inp_files.read()
    inp_files.close()
    out_files = open(enc,"wt", encoding='utf-8')
    for c in i_data:
        if c !='\0':
            out_files.write(c)
    out_files.close()
    
#creating database and inserting files into respectives data tables

conn = sqlite3.connect("medicare_hospital_comapre.db")
curs = conn.cursor()

for file_name in list_files:
    with open(file_name,encoding='utf-8') as f:
        read = csv.reader(f,delimiter=',')
        listings=[]
        for row in read:
            listings.append(row)
        listings[0] = [format_data(z) for z in listings[0]]; 
        cnt_num=len(listings[0])
        listings[0]=[concat(e) for e in listings[0]]
        listings[0] = ','.join(str(o) for o in listings[0])
        table='create table if not exists '+file_name[:-4]+' ('+listings[0]+');'
        rows = conn.execute(table)
        i=1
        j=len(listings)
        if (file_name != 'psi_april2017.csv') and (file_name != 'mort_readm_april2017.csv'):
            while (i<j):
                listings[i]=[removequotes(f) for f in listings[i]]
                innn=conn.execute("insert into "+file_name[:-4]+" values("+ ','.join(str("'"+k+"'") for k in listings[i]) +')')
                i+=1;
        else:
            while (i<(j-2)):
                listings[i]=[removequotes(f) for f in listings[i]]
                innn=conn.execute("insert into "+file_name[:-4]+" values("+ ','.join(str("'"+k+"'") for k in listings[i]) +')')
                i+=1;
            listings[:] = []
conn.commit()
conn.close()

#Defining focused states
states_abbr=['CA','FL','GA','IL','KS','MI','NY','OH','PA','TX']
states_ff=['California','Florida','Georgia','Illinois','Kansas','Michigan','New York','Ohio','Pennsylvania','Texas']
states_dict = dict(zip(states_abbr,states_ff))

#Getting the rankings file and converting it into a dataframe
url_1 = 'http://kevincrook.com/utd/hospital_ranking_focus_states.xlsx'
df = pd.read_excel(url_1,dtype = str)
conn = sqlite3.connect("medicare_hospital_comapre.db")
curs = conn.cursor()

#Quering and joining the two tables and considering the columns of interest
query = ("SELECT * FROM hospital_general_information;")
result = pd.read_sql_query(query,conn)
Nation_Rank = DataFrame.merge(result, df, how='left', left_on= 'provider_id', right_on= 'Provider ID', left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=False, indicator=False)
Sorted_rank = Nation_Rank.reindex(Nation_Rank.Ranking.astype(int).sort_values().index)

#Getting the top 100 ranks nationwide and writing it to excel
Nationwide =Sorted_rank[['provider_id','hospital_name','city','state','county_name']].head(100)
Nationwide.columns=['Provider ID', 'Hospital Name', 'City','State','County']
writer = ExcelWriter('hospital_ranking.xlsx', engine = 'xlsxwriter')
Nationwide.to_excel(writer,'Nationwide', index = False)

#Getting the top 100 ranks statewide for the states of interest and writing it to excel
for s in states_abbr:
    focused_states = Sorted_rank[Sorted_rank['state']== s]
    focused_states1 =focused_states[['provider_id','hospital_name','city','state','county_name']].head(100)
    focused_states1.to_excel(writer, sheet_name = states_dict[s] , index = False)
writer.close()
writer.save()

conn = sqlite3.connect("medicare_hospital_comapre.db")
curs = conn.cursor()

#Quering the table of interest and reading the data
score_query = ("select state, measure_id, measure_name, cast(score as int) score from timely_and_effective_care___hospital where score not in ('Not Available');")
score = pd.read_sql_query(score_query,conn)

#Sorting the data in required order and converting it to a dataframe
nation_score = score.sort_values('measure_id').groupby(['measure_id','measure_name']).agg({'score':['min','max','mean','std']})
nation_score = pd.DataFrame(nation_score.to_records())

#Creating columns and writiing the data to excel nationwide
nation_score.columns=['Measure ID', 'Measure Name', 'Minimum','Maximum','Average','Standard Deviation']
writer = ExcelWriter('measure_statistics.xlsx', engine = 'xlsxwriter')
nation_score.to_excel(writer,'Nationwide', index = False)

#Creating columns and writiing the data to excel statewise
for t in states_abbr:
    states_stat = score[score['state']== t]
    states_stat1 = states_stat.sort_values('measure_id').groupby(['measure_id','measure_name']).agg({'score':['min','max','mean','std']})
    states_stat1 = pd.DataFrame(states_stat1.to_records())
    states_stat1.columns=['Measure ID', 'Measure Name', 'Minimum','Maximum','Average','Standard Deviation']
    states_stat1.to_excel(writer, sheet_name = states_dict[t] , index = False)
writer.close()
writer.save()